In [39]:
import pandas as pd
import sqlite3
import pyodbc

DB = {'servername': r'DESKTOP-R9U7MMK\SQLEXPRESS',
      'database': 'PR4'}

# Constructing the connection string with Windows Authentication
conn_str = (
    'Driver={SQL Server};'
    'SERVER=' + DB['servername'] + ';'
    'DATABASE=' + DB['database'] + ';'
    'Trusted_Connection=yes;'
)

# Establishing the connection
export_conn = pyodbc.connect(conn_str)
export_cursor = export_conn.cursor()
export_cursor

In [55]:
inventory_levels_df = pd.read_csv('GO_SALES_INVENTORY_LEVELSData.csv', index_col=False)
inventory_levels_df

,INVENTORY_YEAR,INVENTORY_MONTH,PRODUCT_NUMBER,INVENTORY_COUNT
0,2021,4,48,1932
1,2021,4,49,1400
2,2021,4,50,21705
3,2021,4,51,9710
4,2021,4,52,5616
...,...,...,...,...
3883,2022,12,111,1128
3884,2022,12,112,7810
3885,2022,12,113,3485
3886,2022,12,114,350


In [58]:
# Read product data from SQLite database into a DataFrame
goSales = sqlite3.connect('go_sales.sqlite')
product_df = pd.read_sql_query("SELECT PRODUCT_NUMBER, MARGIN, LANGUAGE, DESCRIPTION FROM product", goSales)
inventory_levels_df['PRODUCT_NUMBER'] = inventory_levels_df['PRODUCT_NUMBER'].astype(str)

# Merge inventory_levels_df with product_df on PRODUCT_NUMBER
merged_df = pd.merge(inventory_levels_df, product_df, on='PRODUCT_NUMBER', how='left')

# Replace NaN values in MARGIN, LANGUAGE, and DESCRIPTION with empty strings
merged_df[['MARGIN', 'LANGUAGE', 'DESCRIPTION']] = merged_df[['MARGIN', 'LANGUAGE', 'DESCRIPTION']].fillna('')

# Establish connection to SQL Server database
export_conn = pyodbc.connect(conn_str)
cursor = export_conn.cursor()

# Iterate over the rows of the merged DataFrame and insert them into the database
for index, row in merged_df.iterrows():
    cursor.execute("INSERT INTO [PR4].[dbo].[INVENTORY_SALES_LEVELS] (INVENTORY_YEAR, INVENTORY_MONTH, PRODUCT_NUMBER, INVENTORY_COUNT, MARGIN, LANGUAGE, DESCRIPTION) VALUES (?, ?, ?, ?, ?, ?, ?)",
                   row['INVENTORY_YEAR'], row['INVENTORY_MONTH'], row['PRODUCT_NUMBER'], row['INVENTORY_COUNT'], row['MARGIN'], row['LANGUAGE'], row['DESCRIPTION'])

# Commit the transaction and close the cursor and connection
export_conn.commit()
cursor.close()
export_conn.close()


In [67]:
# Establish connection to SQLite database
goSales = sqlite3.connect('go_sales.sqlite')

# Read data from the 'returned_item' table into a DataFrame
returned_item_df = pd.read_sql_query("SELECT RETURN_CODE, RETURN_DATE, ORDER_DETAIL_CODE, RETURN_REASON_CODE, RETURN_QUANTITY FROM returned_item", goSales)

# Establish connection to SQL Server database
export_conn = pyodbc.connect(conn_str)
cursor = export_conn.cursor()

# Iterate over the rows of the DataFrame and insert them into the SQL Server database
for index, row in returned_item_df.iterrows():
    cursor.execute("INSERT INTO [PR4].[dbo].[RETURNED_ITEM] (RETURN_CODE, RETURN_DATE, ORDER_NUMBER, RETURN_REASON_CODE, RETURN_QUANTITY) VALUES (?, ?, ?, ?, ?)",
                   row['RETURN_CODE'], row['RETURN_DATE'], row['ORDER_DETAIL_CODE'], row['RETURN_REASON_CODE'], row['RETURN_QUANTITY'])

# Commit the transaction and close the cursor and connection
export_conn.commit()
cursor.close()
export_conn.close()

In [78]:
# Establish connection to SQLite database
goSales = sqlite3.connect('go_sales.sqlite')

# Read data from the 'SALES_TARGETData' table into a DataFrame
sales_targetdata_df = pd.read_sql_query("SELECT Id, SALES_STAFF_CODE, SALES_YEAR, SALES_PERIOD, RETAILER_NAME, PRODUCT_NUMBER, SALES_TARGET, RETAILER_CODE FROM SALES_TARGETData", goSales)

# Establish connection to SQL Server database
export_conn = pyodbc.connect(conn_str)
cursor = export_conn.cursor()

# Merge with product_df on PRODUCT_NUMBER to get additional columns
merged_sales_targetdata_df = pd.merge(sales_targetdata_df, product_df, on='PRODUCT_NUMBER', how='left')

# Iterate over the rows of the DataFrame and insert them into the SQL Server database
for index, row in merged_sales_targetdata_df.iterrows():
    cursor.execute("INSERT INTO [PR4].[dbo].[SALES_TARGET_DATA] (SALES_TARGET_DATA_id, SALES_STAFF_CODE, SALES_YEAR, SALES_PERIODS, RETAILER_NAME, PRODUCT_NUMBER, SALES_TARGET, RETAILER_CODE, MARGIN, LANGUAGE, DESCRIPTION) VALUES (?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?)",
                   row['Id'], row['SALES_STAFF_CODE'], row['SALES_YEAR'], row['SALES_PERIOD'], row['RETAILER_NAME'], row['PRODUCT_NUMBER'], row['SALES_TARGET'], row['RETAILER_CODE'], row['MARGIN'], row['LANGUAGE'], row['DESCRIPTION'])

# Commit the transaction and close the cursor and connection
export_conn.commit()
cursor.close()
export_conn.close()


KeyboardInterrupt: 